In [11]:
import sys
sys.path.append('../gustav/src/')

from gustav import ncbi, nlm, nih
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': True,
})



matplotlib.rcParams.update({"axes.labelsize": 7,
"xtick.labelsize": 7,
"ytick.labelsize": 7,
"legend.fontsize": 7,
"font.size":7})
matplotlib.rc('font', family='Helvetica') 
matplotlib.rc('pdf', fonttype=42)
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams['axes.unicode_minus'] = False

matplotlib.rcParams['xtick.major.size'] = 2
matplotlib.rcParams['xtick.major.width'] = 0.5
matplotlib.rcParams['xtick.minor.size'] = 2
matplotlib.rcParams['xtick.minor.width'] = 0.5

matplotlib.rcParams['ytick.major.size'] = 2
matplotlib.rcParams['ytick.major.width'] = 0.5
matplotlib.rcParams['ytick.minor.size'] = 2
matplotlib.rcParams['ytick.minor.width'] = 0.5


In [12]:
decisions = pd.read_csv("../models/model_organisms/base.csv").sort_values(['taxon_ncbi', 'Round'])
approved = decisions.loc[decisions['Approved Status'] == 'approved'].drop_duplicates(subset = ['taxon_ncbi'])
approved = approved.sort_values(["taxon_ncbi", "Round"]).groupby("taxon_ncbi").head(1)


notapproved = decisions.loc[~decisions.taxon_ncbi.isin(approved.taxon_ncbi)].sort_values(['taxon_ncbi', 'Round']).drop_duplicates(subset = ['taxon_ncbi'])
notapproved['Round'] = decisions.sort_values("Round").iloc[-1].Round
decisions = pd.concat([approved, notapproved])

In [ ]:
decisions.taxon_ncbi.unique().shape

In [ ]:
names_to_track = []
for organism in tqdm(decisions['taxon_ncbi'].unique(), total = len(decisions['taxon_ncbi'].unique())):
    organism = str(organism)
    names_to_track.append(int(organism))

In [15]:
taxon_names = ncbi.taxonomy('names')
taxon_names['taxon_name'] = taxon_names['taxon_name'].str.lower()
taxon_names = taxon_names.loc[((taxon_names.name_class.str.contains('name')) | (taxon_names.name_class == 'synonym'))]

In [16]:


names_to_track = dict([(taxa, taxon_names.loc[(taxon_names.taxon_ncbi == taxa)]['taxon_name'].to_list()) for taxa in names_to_track])

In [17]:
term_expanded = {
"genome": [
    'chip seq', 'chip sequencing', 'chip-chip', 'chip-seq', 'chip-sequencing',
    'chromatin immunoprecipitation followed by sequencing', 'clinicogenomic',
    'clinicogenomics', 'clip seq', 'clip seq', 'clip sequencing', 'clip sequencing',
    'clip-seq', 'clip-seq', 'clip-sequencing', 'clip-sequencing', 'epigenome',
    'epigenomes', 'epigenomic', 'epigenomics', 'eqtl', 'eqtls', 'exome',
    'expression quantitative trait loci', 'ewas', 'genome', 'genome-scale',
    'genome-wide', 'genomes', 'genomic', 'genomics', 'glycome', 'glycomes',
    'glycomics', 'glycoproteome', 'glycoproteomes', 'glycoproteomic',
    'glycoproteomics', 'gwas', 'high-throughput nucleotide sequencing', 'hits seq',
    'hits sequencing', 'hits-seq', 'hits-sequencing', 'in situ proximity ligation',
    'in-situ proximity ligation', 'interactome', 'interactomes', 'interactomic',
    'interactomics', 'metabolome', 'metabolomes', 'metabolomic', 'metabolomics',
    'metagenome', 'metagenomes', 'metagenomics', 'microarray', 'microarrays',
    'multi-ome', 'multi-omes', 'multi-omic', 'multi-omics', 'multiome', 'multiomes',
    'multiomic', 'multiomics', 'next generation sequencing', 'next generation-sequencing',
    'next-generation sequencing', 'next-generation-sequencing', 'ngs', 'nutrigenome',
    'nutrigenomes', 'nutrigenomic', 'nutrigenomics', 'oligonucleotide array sequence analysis',
    'omics', 'onco-genome', 'onco-genomes', 'onco-genomics', 'onco-genomics',
    'onco-proteogenome', 'onco-proteogenomes', 'onco-proteogenomic', 'onco-proteogenomics',
    'oncogenome', 'oncogenomes', 'oncogenomics', 'oncogenomics', 'oncoproteogenome',
    'oncoproteogenomes', 'oncoproteogenomic', 'oncoproteogenomics', 'par-clip',
    'pharmacogenome', 'pharmacogenomes', 'pharmacogenomic', 'pharmacogenomics',
    'phenome', 'phenomes', 'phenomic', 'phenomics', 'phosphoproteome', 'phosphoproteomes',
    'phosphoproteomic', 'phosphorpoteomics', 'protein array', 'protein array analysis',
    'protein interaction map', 'protein interaction mapping', 'protein interaction maps',
    'protein interaction network', 'protein interaction networks', 'protein–protein interaction map',
    'protein–protein interaction mapping', 'protein–protein interaction maps',
    'protein–protein interaction network', 'protein–protein interaction networks', 'proteogenome',
    'proteogenomes', 'proteogenomic', 'proteogenomics', 'proteome', 'proteomes', 'proteomic',
    'proteomics', 'radiogenome', 'radiogenomes', 'radiogenomic', 'radiogenomics', 'rna seq',
    'rna sequencing', 'rna-seq', 'rna-sequencing', 'rnaseq', 'toxicogenome', 'toxicogenomes',
    'toxicogenomic', 'toxicogenomics', 'transcriptome', 'transcriptomes', 'transcriptomic',
    'transcriptomics', 'wes', 'wgs', 'whole-exome', 'whole-genome']}

In [18]:
pubmed = ncbi.pubmed('main',
                    columns = ['pubmed_id','pubdate'],
                    filters = {"pubdate": np.arange(1990, 2021)})

### we load the matrix of PubMed that checked for organisms (see README; this is done separately due to computational resources)

In [19]:
nhgri = np.load("../cache/all_combined_nhgri_organisms.npy")
nhgri_df = pd.DataFrame(nhgri, columns = names_to_track.keys())
del nhgri


In [28]:
genomics = np.load('../cache/all_combined_genome.npy')
genomics_df = pd.DataFrame(genomics, columns = ["genomics"])
del genomics

In [20]:
combined = nhgri_df
del nhgri_df

In [21]:
combined['pubdate'] = pubmed['pubdate'].reset_index(drop = True)

In [24]:
def year_count(df, organism):
    
    return df.groupby(df.pubdate).agg({organism:np.sum})

def get_papers_that_have_term_and_organism(terms_df, organisms_df, term, organism):
    numerator = np.bitwise_and(terms_df[term], organisms_df[organism])
    
    return numerator

def get_genome_share(organism, years, group = False, rolling = False):
    
    total_mesh_per_year = year_count(combined, organism).reset_index()
    
    total_mesh_per_year = total_mesh_per_year.loc[total_mesh_per_year.pubdate.isin(years)]
    
    temp_term = get_papers_that_have_term_and_organism(genomics_df, \
                            combined, "genomics", organism).to_frame()
    temp_term.columns = [organism]
    temp_term['pubdate'] = pubmed['pubdate'].reset_index(drop= True)
    
    percentage = year_count(temp_term, organism).reset_index()
    percentage = percentage.loc[percentage.pubdate.isin(years)]
    
    if rolling:
        total_mesh_per_year[organism] = total_mesh_per_year[organism].rolling(3).sum()
        percentage[organism] = percentage[organism].rolling(3).sum()
        
        total_mesh_per_year = total_mesh_per_year.iloc[2:]
        percentage = percentage.iloc[2:]
    if group:
        total_mesh_per_year = total_mesh_per_year.loc[total_mesh_per_year.pubdate != years[0] + 10].groupby(np.arange(len(total_mesh_per_year.loc[total_mesh_per_year.pubdate != years[0] + 10].index))//5, axis=0).sum().reset_index(drop = True)
        percentage = percentage.loc[percentage.pubdate != years[0] + 10].groupby(np.arange(len(percentage.loc[percentage.pubdate != years[0] + 10].index))//5, axis=0).sum().reset_index(drop = True)
        

    percentage[organism] = percentage[organism] / total_mesh_per_year[organism]
    percentage[organism] = percentage[organism].fillna(0)    
    return percentage

def type_of_organism(x):
    if x in decisions.loc[decisions['Approved Status'] == 'not approved']['taxon_ncbi'].to_list():
        return "not approved"
    elif x in decisions.loc[decisions['Approved Status'] == "approved"]['taxon_ncbi'].to_list():
        return "approved"
    elif x in genomes_25for25:
        return "sanger"
    elif x in genomes_nature_unsequenced:
        return "nature"
    else:
        return None

In [ ]:
to_plot = []
for i in tqdm(combined.columns, total = len(combined.columns)):

	if i != 'pubdate':
		# try:
		total_mesh_per_year = year_count(combined, i).reset_index()

		if type_of_organism(i) == "not approved" or type_of_organism(i) == "approved":
			round_year = pd.to_datetime(decisions.loc[decisions.taxon_ncbi == i].iloc[0]['Round']).year
			total_mesh_per_year = total_mesh_per_year.loc[total_mesh_per_year['pubdate'].isin(range(round_year - 10, round_year + 11))]

			genome_share = get_genome_share(i, range(round_year - 10, round_year + 11), group = True)
		else:
				
			total_mesh_per_year = total_mesh_per_year.loc[total_mesh_per_year['pubdate'].isin(np.arange(1996, 2017))]
	
			genome_share = get_genome_share(i, np.arange(1996, 2017), group = True)
		genome_share = genome_share.reset_index(drop = True)
		del genome_share['pubdate']

		to_plot.append(genome_share)
		
		# except KeyError:
		# 	print(i)

In [ ]:
to_plot = pd.concat(to_plot, axis = 1)
# to_plot_25for25['pubdate'] = range(-8, 11)
to_plot['pubdate'] = ['-10 to -5', '-5 to -1', '1 to 5', '6 to 10']
to_plot['relative'] = '1995to2015'
to_plot = to_plot.melt(id_vars = ["pubdate", "relative"])
to_plot['taxon_ncbi'] = to_plot['variable']
del to_plot['variable']

In [47]:
from scipy.stats import mannwhitneyu, wilcoxon
from statsmodels.stats.multitest import multipletests
from scipy.stats import fisher_exact

In [34]:
to_plot['type'] = to_plot['taxon_ncbi'].apply(type_of_organism)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (3, 2), dpi = 300 )
flierprops = dict(markerfacecolor='k', markersize=1,
              marker = 'o')
# sns.boxplot(to_plot.loc[~((to_plot['type'] == 'sanger') & (to_plot['pubdate'] != '6 to 10')) & (to_plot['type'] != "nature")].sort_values(by = ['pubdate', 'type']), x = 'pubdate', y = 'value',\
#             ax = ax, notch = True, hue = 'type', flierprops = flierprops \
#             , boxprops = {"edgecolor": "black", 'linewidth': 0.5}, \
#             medianprops = {'color':'black', 'linewidth':0.5}, whiskerprops={'color':'black', 'linewidth': 0.5}, capprops={'color':'black', 'linewidth': 0.5},
#            palette = ["#9CADCE", "#F37668", "#8CBA80"])
sns.boxplot(to_plot.loc[~((to_plot['type'] == 'sanger') ) & (to_plot['type'] != "nature")].sort_values(by = ['pubdate', 'type']), x = 'pubdate', y = 'value',\
            ax = ax, notch = False, hue = 'type', flierprops = flierprops \
            , boxprops = {"edgecolor": "black", 'linewidth': 0.5}, \
            medianprops = {'color':'black', 'linewidth':0.5}, whiskerprops={'color':'black', 'linewidth': 0.5}, capprops={'color':'black', 'linewidth': 0.5},
           palette = ["#9CADCE", "#F37668", "#8CBA80"])
# sns.stripplot(to_plot.loc[to_plot['type'] != "nature"].sort_values(by = ['pubdate', 'type']), x = 'pubdate', y = 'value', hue = 'type',\
#             ax = ax, jitter=0.4, marker='o', size =2.5, alpha=0.4, edgecolor='black', linewidth=0.5, dodge=True, )

ax.set_ylabel('Share of Genomics')
ax.set_xlabel('Relative Year')
# ax.set_xticks(np.arange(-10, 11, 2))
# ax.set_title(f"N={len(to_plot['value'].unique())}", fontsize = 20)
ax.get_legend().remove()
ax.tick_params(axis = "both", colors = "k")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_linewidth(0.5)
ax.spines['left'].set_linewidth(0.5)
# ax.legend(title='Status', loc='upper right', bbox_to_anchor=(1.2, 0.6))
plt.show()

In [ ]:
mannwhitneyu(forstats.loc[(forstats['type'] == "approved") & (forstats['pubdate'] == '1 to 5')]['value'].iloc[0],
             forstats.loc[(forstats['type'] == "not approved") & (forstats['pubdate'] == '1 to 5')]['value'].iloc[0])

In [ ]:
mannwhitneyu(forstats.loc[(forstats['type'] == "approved") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0],
             forstats.loc[(forstats['type'] == "not approved") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0])

In [ ]:
mannwhitneyu(forstats.loc[(forstats['type'] == "approved") & (forstats['pubdate'] == '-5 to -1')]['value'].iloc[0],
             forstats.loc[(forstats['type'] == "not approved") & (forstats['pubdate'] == '-5 to -1')]['value'].iloc[0])

In [ ]:
mannwhitneyu(forstats.loc[(forstats['type'] == "approved") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0],
             forstats.loc[(forstats['type'] == "not approved") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0])

In [ ]:
mannwhitneyu(forstats.loc[(forstats['type'] == "approved") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0],
             forstats.loc[(forstats['type'] == "sanger") & (forstats['pubdate'] == '6 to 10')]['value'].iloc[0])

In [36]:
taxon_nodes = ncbi.taxonomy('nodes')
taxon_names = ncbi.taxonomy('names')
taxon_names['lower_taxon_name'] = taxon_names['taxon_name'].str.lower()

In [ ]:
phyla = ["Bryozoa", "Phoronida", "Brachiopoda", "Annelida","Orthonectida",\
         "Dicyemida","Mollusca", "Rhombozoa", "Platyhelminthes", "Acanthocephala",\
         "Rotifera", "Arthropoda", "Onychophora", "Nematoda", \
         "Tardigrada", "Priapulida", "Echinodermata", "Hemichordata", \
         "Chordata", "Xenacoelomorpha", "Cnidaria", "Placozoa", \
         "Porifera", "Ctenophora", "Chaetognatha", "Cycliophora", "Entoprocta", \
        "Gastrotricha", "Gnathostomulida", "Kinorhyncha", "Loricifera", "Micrognathozoa",\
        "Nematomorpha", "Sipuncula"]

phyla = [i.lower() for i in phyla]
 
phyla_ncbi = [10205, 120557,7568, 6340, 33209,10215, 6447, 6217,  6157, 10232, 10190, 6656, 27563,\
             6231, 42241, 33467, 7586, 10219, 7711, 1312402, 6073, 10226, 6040, 10197, 10229, 69815, \
             43120, 33313, 66780,51516, 310840, 195505, 33310, 6433]
# dicyemida is also known as Rhombozoa
num_species = [10491, 10, 443, 17210, 43, None,  117358, 123, 29285, 1194, 1583, 1242040, 182, 24783, 1157, 19,\
               20509, 120, 81683, 395, 10105, 1, 8346, 242, 186, 2, 169, 790, 109, 179, 30, 1, 351, 1507 ]
len(phyla) == len(phyla_ncbi) == len(num_species)

In [38]:
def find_which_phyla_the_organism_belongs_under(taxon_ncbi, rank):
    
    parent = taxon_nodes.loc[taxon_nodes.taxon_ncbi == taxon_ncbi].iloc[0]
    
    if parent['rank'] != rank:
#         print(parent.taxon_ncbi, parent['rank'])
        parent = find_which_phyla_the_organism_belongs_under(parent.parent_taxon_ncbi, rank)
    
    return parent

In [ ]:

which_phyla = []
for taxon_ncbi in tqdm(decisions.taxon_ncbi):
    # skip Thecamonas trahens 	
    try:
        parent = find_which_phyla_the_organism_belongs_under(taxon_ncbi, 'kingdom')
    except RecursionError:
        try:
            parent = find_which_phyla_the_organism_belongs_under(taxon_ncbi, "phylum")
            which_phyla.append(("no kingdom", taxon_ncbi, parent.taxon_ncbi,  taxon_names.loc[(taxon_names.taxon_ncbi == parent.taxon_ncbi)  & (taxon_names.name_class == 'scientific name')].iloc[0].lower_taxon_name))
        except RecursionError:
            which_phyla.append(("unranked", taxon_ncbi, "unranked", "unranked"))
            continue
    parent_rank = taxon_names.loc[(taxon_names.taxon_ncbi == parent.taxon_ncbi)].iloc[0].lower_taxon_name
    if parent_rank == 'animalia':
        parent = find_which_phyla_the_organism_belongs_under(taxon_ncbi, 'phylum')
        which_phyla.append(("animalia", taxon_ncbi, parent.taxon_ncbi, taxon_names.loc[(taxon_names.taxon_ncbi == parent.taxon_ncbi)  & (taxon_names.name_class == 'scientific name')].iloc[0].lower_taxon_name))
    else:
        parent = find_which_phyla_the_organism_belongs_under(taxon_ncbi, "phylum")
        which_phyla.append((parent_rank, taxon_ncbi, parent.taxon_ncbi,  taxon_names.loc[(taxon_names.taxon_ncbi == parent.taxon_ncbi)  & (taxon_names.name_class == 'scientific name')].iloc[0].lower_taxon_name))
        
    

In [40]:
which_phyla_df = pd.DataFrame(which_phyla, columns = ['kingdom', 'taxon_ncbi', 'phylum_taxon', 'name'])

In [41]:
which_phyla_df['decision'] = which_phyla_df['taxon_ncbi'].map(decisions.set_index("taxon_ncbi")['Approved Status'].to_dict())

In [42]:
all_df = which_phyla_df
all_df['taxon_ncbi'] = all_df['taxon_ncbi'].astype(int)

# map_count = dict(zip(phyla_ncbi, num_species))
# animalia['species_count'] = animalia['phylum_taxon'].map(map_count)

all_df['kingdom'] = all_df['kingdom'].map({"animalia": "animalia", "fungi": "fungi", "oomycetes": "oomycetes"})
all_df['kingdom'] = all_df['kingdom'].fillna("rest")

### the number of species below are sourced from Z.-Q. Zhang, Animal Biodiversity: An Outline of Higher-Level Classification and Survey of Taxonomic Richness (Magnolia Press, 2011) and references within.

In [43]:
map_count = {'bryozoa': 10491,
 'phoronida': 10,
 'brachiopoda': 443,
 'annelida': 17210,
 'orthonectida': 43,
 'dicyemida': None,
 'mollusca': 117358,
 'rhombozoa': 123,
 'platyhelminthes': 29285,
 'acanthocephala': 1194,
 'rotifera': 1583,
 'arthropoda': 1242040,
 'onychophora': 182,
 'nematoda': 24783,
 'tardigrada': 1157,
 'priapulida': 19,
 'echinodermata': 20509,
 'hemichordata': 120,
 'chordata': 81683,
 'xenacoelomorpha': 395,
 'cnidaria': 10105,
 'placozoa': 1,
 'porifera': 8346,
 'ctenophora': 242,
 'chaetognatha': 186,
 'cycliophora': 2,
 'entoprocta': 169,
 'gastrotricha': 790,
 'gnathostomulida': 109,
 'kinorhyncha': 179,
 'loricifera': 30,
 'micrognathozoa': 1,
 'nematomorpha': 351,
 'sipuncula': 1507}

In [45]:
def rename(x):
    if x['kingdom'] == "animalia":
        return x['name']
    elif x['kingdom'] == "fungi":
        return x['name']
    elif x['kingdom'] == "oomycetes":
        return x['name']
    else:
        return "other"

all_df['groups'] = all_df.apply(lambda x: rename(x), axis = 1)

In [ ]:

# all_df = all_df.loc[all_df.decision == "approved"]
ratios = []
for phyla in which_phyla_df['groups'].unique():
    print(phyla)

    
    a1 = all_df.loc[(all_df['groups'] == phyla)].shape[0]
    a3 = all_df.loc[(all_df['groups'] == phyla) & (all_df['decision'] == 'approved')].shape[0]
    b1 = all_df.loc[(all_df['groups'] != phyla)].shape[0]

    if phyla in map_count:
        a2 = map_count[phyla]
        b2 = sum([i for i in map_count.values() if i is not None]) - b1
    
        # b1 = all_df.loc[(all_df['groups'] != phyla) & (all_df['decision'] == 'approved')].shape[0]
        # b2 = all_df.loc[(all_df['groups'] != phyla) & (all_df['decision'] == 'not approved')].shape[0]
        print(a1, a2, b1, b2)
     
        ratio, p_value = fisher_exact([[a1, a2], [b1, b2]])
        ratios.append((phyla, np.log2(ratio), p_value, a1, a3))
        print(np.log(ratio), p_value)
    else:
        continue

In [ ]:
ratios_df = pd.DataFrame(ratios, columns = ["phyla", "ratio", "pvalue", "total", "approved"])
ratios_df['phyla'] = ratios_df['phyla'].apply(lambda x: x.title())
ratios_df = ratios_df.sort_values("phyla", ascending = False)
ratios_df['color'] = ratios_df['ratio'].apply(lambda x: "#C83E4D" if x< 0 else "#3ABEFF")

fig, ax1 = plt.subplots(1, 1, figsize = (4, 3), dpi = 300 )

ax1.barh(ratios_df['phyla'], ratios_df['total'], color = '#C83E4D', linewidth = 0) 
ax1.barh(ratios_df['phyla'], ratios_df['approved'], color = '#3ABEFF', linewidth = 0) 
# ax1.barh(ratios_df['phyla'], ratios_df['ratio'], color = ratios_df['color']) 
# ax1.barh(ratios_df.loc[ratios_df.ratio  > 0]['phyla'], ratios_df.loc[ratios_df.ratio  > 0]['ratio'], color = "#3ABEFF")
# ax1.set_xlim(-50, 70)
# ax1.set_xticklabels([60, 40, 20, 0, 20, 40, 60])
ax1.axvline(0, c = 'k', linewidth = 0.5)
ax1.set_xlabel('Total submissions' ,color = 'k')
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_linewidth(0.5)
ax1.spines['left'].set_linewidth(0.5)
ax1.tick_params(axis = "both", colors = "k")
plt.show()
# plt.show()